In [1]:
import numpy as np 
import matplotlib.pyplot as plt

'''
To do:
    You may need to change draft to meet the requirement in P1
'''

def data_loader(file):
    a = np.genfromtxt(file, delimiter=',', skip_header=0)
    x = a[:, 1:] / 255.0
    y = a[:, 0]
    return (x, y)

x_train, y_train = data_loader('mnist_train.csv')
print('data loading done')


data loading done


In [32]:
test_labels = [1,8]
indices = np.where(np.isin(y_train,test_labels))[0]


x = x_train[indices]
y = y_train[indices]

y[y == test_labels[0]] = 0
y[y == test_labels[1]] = 1


# open a (new) file to write
outF = open("features.txt", "w")
x3 = x[2]

print(x.shape)

for i in range(len(x3)):
    outF.write(str(round(x3[i], 2)))
    if i != len(x3) - 1:
        outF.write(",")
outF.close()



(12593, 784)


In [33]:
num_epochs = 500
m = x.shape[1]
alpha = 0.1


w = np.random.rand(m)

b = np.random.rand()




In [34]:
c = np.zeros(num_epochs)


for epoch in range(num_epochs):
    a = x@w + b
    a = (a >= 0)
    
    w -= alpha * (x.T)@(a-y)
    b -= alpha * (a-y).sum()
    

In [35]:
# open a (new) file to write
outF = open("weights.txt", "w")

for i in range(len(w)):
    outF.write(str(round(w[i], 4)))
    outF.write(",")
outF.write(str(round(b, 4)))
outF.close()

In [36]:
import math

# Todo: load and measure the given sample
new_test = np.loadtxt('test.txt', delimiter=',')
new_x = new_test / 255.0





outF = open("activation_values.txt", "w")
for i in range(len(new_x)):
    value = -(np.dot(new_x[i],w)+b)
    if value > 700:
        value = 700
    if value < -700:
        value = -700
    ai = 1/(1+math.exp(value))
    outF.write(str(round(ai, 2)))
    if i != len(new_x) - 1:
        outF.write(",")
outF.close()
    
outF = open("predictions.txt", "w")
for i in range(len(new_x)):
    value = -(np.dot(new_x[i],w)+b)
    if value > 700:
        value = 700
    if value < -700:
        value = -700
    ai = 1/(1+math.exp(value))
    outF.write(str(round(ai)))
    if i != len(new_x) - 1:
        outF.write(",")
outF.close()



In [44]:
# num_hidden_uints should be 28 in P1
num_hidden_uints = 28

# sigmoid should be used here
def sigmoid(x):
    for i in range(len(x)):
        x_ori = x[i] 
        if x_ori > 700:
            x_ori = 700
        if x_ori < -700:
            x_ori = -700
        x_log = 1.0 / (1.0 + math.exp(-x_ori))
        x[i] = x_log
    return x

def sigmoid_single(x):
    x = 1.0 / (1.0 + math.exp(-x))
    return x




def nnet(train_x, train_y, lr, num_epochs):
    num_train = len(train_y)
    
    train_x = np.hstack((train_x, np.ones(num_train).reshape(-1,1)))
    
    num_input_uints = train_x.shape[1]  # 28
    
    
    wih = np.random.uniform(low=-1, high=1, size=(num_hidden_uints, num_input_uints)) 
    who = np.random.uniform(low=-1, high=1, size=(1, num_hidden_uints+1)) 
    
    for epoch in range(1, num_epochs+1):
        out_o = np.zeros(num_train)
        out_h = np.zeros((num_train, num_hidden_uints+1))  
        out_h[:,-1] = 1
        for ind in range(num_train):
            row = train_x[ind]  # len = 28 
            out_h[ind, :-1] = sigmoid(np.matmul(wih, row))
            out_o[ind] = 1 / (1 + np.exp(-sum(out_h[ind] @ who.T)))
            
            delta = np.multiply(sigmoid(out_h[ind]), (train_y[ind] - out_o[ind]) * np.squeeze(who))
            wih += lr * np.matmul(np.expand_dims(delta[:-1], axis=1), np.expand_dims(row,axis=0))
            who += np.expand_dims(lr * (train_y[ind] - out_o[ind]) * out_h[ind,:], axis=0)
        error = sum(- train_y * np.log(out_o) - (1-train_y) * np.log(1-out_o))
        num_correct = sum((out_o > 0.5).astype(int) == train_y)
        
        print('epoch = ', epoch, ' error = {:.7}'.format(error), 'correctly classified = {:.4%}'.format(num_correct / num_train))
    
    return wih.T, who




# Todo: change these hyper parameters
lr = 0.01
num_epochs = 5

W1, W2 = nnet(x, y, lr, num_epochs)

print(W1.shape)
print(W2.shape)



epoch =  1  error = 1280.657 correctly classified = 96.5695%
epoch =  2  error = 899.4157 correctly classified = 97.5780%
epoch =  3  error = 772.9386 correctly classified = 98.0465%
epoch =  4  error = 692.399 correctly classified = 98.2530%
epoch =  5  error = 644.7929 correctly classified = 98.4356%
(785, 28)
(1, 29)


In [45]:
outF = open("first_layer_weights.txt", "w")
for i in range(len(W1)):
    for j in range(len(W1[i])):
        outF.write(str(round(W1[i][j],4)))
        if j != len(W1[i]) - 1:
            outF.write(",")
    outF.write('\n')
outF.close()

In [46]:
outF = open("second_layer_weights.txt", "w")
for i in range(len(W2)):
    for j in range(len(W2[i])):
        outF.write(str(round(W2[i][j],4)))
        if j != len(W2[i]) - 1:
            outF.write(",")
    outF.write('\n')
outF.close()



In [47]:
# Todo: load and measure the given sample
new_test = np.loadtxt('test.txt', delimiter=',')
new_x = new_test / 255.0

W2 = np.squeeze(W2)


outF = open("second_layer_activation_values.txt", "w")
for i in range(len(new_x)):
    t = np.append(new_x[i], [1], axis=0)
    value_first_l = np.dot(t,W1)
    out_first_l = sigmoid(value_first_l)
    out_first_l_bias = np.append(out_first_l, [1], axis=0)
    before_a = np.dot(out_first_l_bias, W2)
    
    after_a = sigmoid_single(before_a)
    outF.write(str(round(after_a, 2)))
    if i != len(new_x) - 1:
        outF.write(",")
outF.close()


In [48]:
outF = open("predictions_NN.txt", "w")
for i in range(len(new_x)):
    t = np.append(new_x[i], [1], axis=0)
    value_first_l = np.dot(t,W1)
    out_first_l = sigmoid(value_first_l)
    out_first_l_bias = np.append(out_first_l, [1], axis=0)
    before_a = np.dot(out_first_l_bias, W2)
    
    after_a = sigmoid_single(before_a)
    outF.write(str(round(after_a)))
    if i != len(new_x) - 1:
        outF.write(",")
outF.close()

In [50]:
# Todo: load and measure the given sample
wrong_fs = np.loadtxt('FNN.txt', delimiter=',')


outF = open("wrong_features.txt", "w")
for i in range(len(wrong_fs)):
    outF.write(str(round(wrong_fs[i], 2)))
    if i != len(wrong_fs) - 1:
        outF.write(",")
outF.close()
